In [1]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import pandas as pd
import os

In [2]:
ruta_entrada= os.path.join( '.','data')
ruta_salida = os.path.join( '.','output')
ciudad = 'Valledupar'
fuente = 'IDEAM'
ruta_serie= os.path.join(ruta_entrada,ciudad,fuente+'.csv')
serie= pd.read_csv(ruta_serie, sep = ';',
                encoding='latin-1')

## Punto 1

In [3]:
def ENFICC(serie_horaria):  #Cálculo ENFICC 99%
    agnos=sorted(serie_horaria['AÑO'].unique())
    ghi_df=pd.DataFrame(columns=["AÑO","MES","DIAS","GHImes"])
    energia = list(serie_horaria)[-1]
    for agno in agnos:
        for mes in range(1,13):
            dias = sorted(serie_horaria[(serie_horaria['AÑO'] == agno) &
                        (serie_horaria['MES'] == mes)]["DIA"].unique())
            ghi_mes = serie_horaria.loc[((serie_horaria["AÑO"] == agno)
                            & (serie_horaria["MES"] == mes)),
                                        energia].sum()/1000
            ghi_dict = {"AÑO":agno, "MES":mes, "DIAS":dias[-1],
                        "GHImes":ghi_mes}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
        
        
    agnos2=sorted(ghi_df['AÑO'].unique())
    df_dias=pd.DataFrame(columns=["AÑO","MES","GHIdia"])
    for agno in agnos2:
        for mes in range(1,13):
            dia=ghi_df.loc[((ghi_df['AÑO']==agno) &
                                   (ghi_df['MES']==mes)),'DIAS'].item()
            ghi_diario=ghi_df.loc[((ghi_df['AÑO']==agno) & 
                                          (ghi_df['MES']==mes)),'GHImes'].item()/dia
            ghi_dict = {'AÑO':agno,'MES':mes,'GHIdia':ghi_diario}
            df_dias=df_dias.append(ghi_dict,ignore_index=True)
        
    ENFICC=df_dias.quantile(0.01,numeric_only=True)['GHIdia']
        
    return print('La ENFICC 99% es', ENFICC)

In [4]:
(serie)

La ENFICC 99% es 4.56900737661


## Punto 3

In [5]:
corta=serie[serie['AÑO'] == 2010] #Df con todo el año 2010
seriecorta2=serie[(serie['AÑO'] >= 2007) & (serie['AÑO'] <=2012 )] #DF con año 2007 al 2011

In [6]:
ciudad = 'Valledupar'
fuente2 = 'NREL'
ruta_serie= os.path.join(ruta_entrada,ciudad,fuente2+'.csv')
serienrel= pd.read_csv(ruta_serie, sep = ';',
                encoding='latin-1')

In [64]:
def MCP(serie_corta,serie_larga):
    serie_ajustada=pd.DataFrame(columns=['AÑO','MES','DIA','HORA','GHI'])
    ghi_obj=serie_corta['IDEAM'].sum()/len(serie_corta)
    sigma_obj=serie_corta['IDEAM'].std()
    serie_larga_comun=serie_larga[(serie_larga['AÑO']>=(serie_corta.iloc[0,0])) & (serie_larga['AÑO']<=(serie_corta.iloc[-1,0]))]
    ghi_ref=serie_larga_comun['NREL'].sum()/len(serie_larga_comun)
    sigma_ref=serie_larga_comun['NREL'].std()
    for i in range(0,len(serie_larga)):
        ghi_i=((ghi_obj-(sigma_obj/sigma_ref)*ghi_ref)+(sigma_obj/sigma_ref)*serie_larga.iloc[i,4]) #COMO HACERLO SIN QUE SE BORRE EL i anterior
    return ghi_i